In [1]:
import tensorflow as tf
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

In [3]:
#Reading data set into df
df = pd.read_csv('data_set.csv')

In [4]:
#exiting unwanted columns
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

In [5]:
#Replacing gender text with 0's and 1's
df.Gender.replace({'Male':0, 'Female':1}, inplace=True)

In [6]:
from sklearn.preprocessing import MinMaxScaler

In [8]:
#Scaling the following columns to values in range [0,1]
scaling = ['Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary', 'CreditScore']
df[scaling] = MinMaxScaler().fit_transform(df[scaling])

In [10]:
#extrapolating the values in column Geography to its own columns with values 0 and 1 
df_set = pd.get_dummies(data=df, columns=['Geography'])

In [11]:
#Differentating target columns (column to be predicted) into Y and data column to X
Y = df_set['Exited']
X = df_set.drop(columns=['Exited'])

In [12]:
#Splitting data randomly to train portion and test portion
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train , Y_test = train_test_split(X,Y,test_size=0.2,random_state = 7)

In [13]:
#Creating a sequential model Architecture
model_new = tf.keras.Sequential([
    tf.keras.layers.Dense(14, input_shape = (12,), activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(18, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [14]:
model_new.compile(
    optimizer='adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [15]:
%load_ext tensorboard

In [16]:
from tensorflow.keras.callbacks import TensorBoard

In [17]:
log_folder = 'logs'
callbacks = TensorBoard(log_dir=log_folder, histogram_freq=1)

In [18]:
#Training Model
model_new.fit(X_train,Y_train, epochs=200, callbacks=callbacks)

Epoch 1/200
250/250 [==============================] - 4s 5ms/step - loss: 0.5200 - accuracy: 0.7966
Epoch 2/200
250/250 [==============================] - 1s 5ms/step - loss: 0.4727 - accuracy: 0.7970
Epoch 3/200
250/250 [==============================] - 1s 5ms/step - loss: 0.4513 - accuracy: 0.8036
Epoch 4/200
250/250 [==============================] - 1s 5ms/step - loss: 0.4365 - accuracy: 0.8079
Epoch 5/200
250/250 [==============================] - 1s 5ms/step - loss: 0.4289 - accuracy: 0.8144
Epoch 6/200
250/250 [==============================] - 1s 4ms/step - loss: 0.4233 - accuracy: 0.8155
Epoch 7/200
250/250 [==============================] - 1s 5ms/step - loss: 0.4152 - accuracy: 0.8198
Epoch 8/200
250/250 [==============================] - 1s 4ms/step - loss: 0.4094 - accuracy: 0.8221
Epoch 9/200
250/250 [==============================] - 1s 4ms/step - loss: 0.4008 - accuracy: 0.8278
Epoch 10/200
250/250 [==============================] - 1s 5ms/step - loss: 0.3942 - accura

In [19]:
model_new.evaluate(X_test,Y_test)

63/63 [==============================] - 1s 4ms/step - loss: 0.3543 - accuracy: 0.8470


[0.3542954921722412, 0.847000002861023]

In [ ]:
model_new.save('weight_model.h5')

In [20]:
Y_predicted = model_new.predict(X_test)
Y_pred = np.round(Y_predicted)

63/63 [==============================] - 1s 3ms/step


In [21]:
Y_Truth = Y_test.to_numpy()

In [22]:
from sklearn.metrics import classification_report

In [23]:
print(classification_report(Y_Truth, Y_pred))

              precision    recall  f1-score   support

           0       0.88      0.93      0.91      1589
           1       0.66      0.53      0.59       411

    accuracy                           0.85      2000
   macro avg       0.77      0.73      0.75      2000
weighted avg       0.84      0.85      0.84      2000

